In [ ]:
from IPython.display import clear_output
!git clone https://github.com/jroakes/querycat.git
!pip install -r querycat/requirements_colab.txt

clear_output()
print('All loaded.')

In [ ]:
import querycat
import searchconsole
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
account = searchconsole.authenticate(client_config='client_secrets.json',
                                     credentials='credentials.json')
webproperty = account['https://www.example.com/']
report = webproperty.query.range('today',days=-30).dimension('date','query','page','country').get().to_dataframe()
#print(report.rows)

In [ ]:
#report=report[report['position'] <= 30]  
report['diff'] = 1

In [ ]:
catz = querycat.Categorize(report, 'query', min_support=10,  alg='apriori')
catz.counts.head(20)

In [ ]:
labels = ['1 to 3', '3 to 5', '5 to 9', '9 to 30']
report['position_bins'] = pd.qcut(report['position'], q=4,labels=labels)

In [ ]:
bsim = querycat.BERTSim()

In [ ]:
bsim.read_df(report, term_col = 'category', diff_col = 'diff')

In [ ]:
bsim.diff_plot(reduction='umap')

In [ ]:
data = {'category': bsim.terms,'embedding_1': bsim.embeddings_2d[:,0], 'embedding_2':bsim.embeddings_2d[:,1]}
embeddings_df=pd.DataFrame(data=data)

In [ ]:
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters = i, init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
    kmeans.fit(bsim.embeddings_2d)
    wcss.append(kmeans.inertia_)

In [ ]:
plt.plot(range(1, 11), wcss)
plt.title('The elbow method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS') #within cluster sum of squares
plt.show()

In [ ]:
n_clusters=input('insert number of topics   ')
print('')
print('')
#Applying kmeans to the dataset / Creating the kmeans classifier
kmeans = KMeans(n_clusters = int(n_clusters), init = 'k-means++', max_iter = 300, n_init = 10, random_state = 0)
y_kmeans = kmeans.fit_predict(bsim.embeddings_2d)
y_kmeans_to_merge=pd.DataFrame(y_kmeans).rename(columns={0:"topic"})

In [ ]:
embeddings_df=embeddings_df.join(y_kmeans_to_merge)

In [ ]:
embeddings_df.drop(columns=['embedding_1','embedding_2'],inplace=True)

In [ ]:
report_export= report.merge(embeddings_df, on='category',how='outer')

### Navigational
Brand, product name, previous touch-point (existing exposure) 

### Informational
info, more information, information, details, accessories, details, features, benefits

### Commercial Investigation
small, medium, large, kids, mens, womens, black, blue, gray, orange, pink, red, green, purple, versus, vs, best, price, pricing, reviews, deals
### Transactional
buy, purchase, sale, coupon, discount


In [ ]:
mydict = {'maytronics' : 'navigational',
            'dolphin' : 'navigational',
            'wave' : 'navigational',
            'what' : 'informational',
            'why ': 'informational',
            'how': 'informational',
            'info': 'informational', 
            'more information': 'informational', 
            'information': 'informational', 
            'details': 'informational', 
            'accessories': 'informational', 
            'details': 'informational', 
            'features': 'informational', 
            'benefits': 'informational',
            'small': 'comercial', 
            'medium': 'comercial', 
            'large': 'comercial', 
            'kids': 'comercial', 
            'mens': 'comercial', 
            'womens': 'comercial', 
            'black': 'comercial', 
            'blue': 'comercial', 
            'gray': 'comercial', 
            'orange': 'comercial', 
            'pink': 'comercial', 
            'red': 'comercial', 
            'green': 'comercial', 
            'purple': 'comercial', 
            'versus': 'comercial', 
            'vs': 'comercial', 
            'best': 'comercial', 
            'price': 'comercial',
            'cost': 'comercial',
            'pricing': 'comercial', 
            'reviews': 'comercial', 
            'deals': 'comercial',         
            'buy': 'transactional',
            'purchase': 'transactional',
            'sale': 'transactional',
            'coupon': 'transactional',
            'discount': 'transactional'}

pat = '|'.join(r"\b{}\b".format(x) for x in mydict.keys())
report_export['intent'] = report_export['query'].str.extract('('+ pat + ')', expand=False).map(mydict)

In [ ]:
report_export.drop(columns=['diff'],inplace=True)

In [ ]:
report_export.to_csv('report_export.csv')